#Perceptron example

In [2]:
import numpy as np
class perceptron():   #A
    def __init__(self, X,y, threshold = 0.5, learning_rate = 0.1, max_epochs = 10): #B
        self.threshold = threshold  #C
        self.learning_rate = learning_rate  #D
        self.X = X  #E
        self.y = y  #E
        self.max_epochs = max_epochs  #F

    def initialize(self, init_type = 'zeros'):  #G
        if init_type == 'random':  
            self.weights = np.random.rand(len(self.X[0])) * 0.05 
        if init_type == 'zeros':   
            self.weights = np.zeros(len(self.X[0])) 

    def train(self):   #H
        epoch = 0 #I
        while True: #J
            error_count = 0   #K
            epoch += 1 #L
            for (X,y) in zip(self.X, self.y): #M
                error_count += self.train_observation(X,y,error_count) 
            if error_count == 0: #N
                print "training succesfull"
                break 
            if epoch >= self.max_epochs: #O
                print "reached maximum epochs, no perfect prediction"
                break

    def train_observation(self,X,y, error_count):   #P
        result = np.dot(X, self.weights) > self.threshold #Q
        error = y - result #R

        if error != 0:   #S
            error_count += 1 #T
            for index, value in enumerate(X): #U
                self.weights[index] +=  self.learning_rate * error * value #V
        return error_count #W

    def predict(self, X):  #X
        return int(np.dot(X, self.weights) > self.threshold) #Y

X = [(1,0,0),(1,1,0),(1,1,1),(1,1,1),(1,0,1),(1,0,1)] #Z
y = [1,1,0,0,1,1]#AA

p = perceptron(X,y) #AB
p.initialize() #AC
p.train() #AD
print p.predict((1,1,1)) #AE
print p.predict((1,0,1)) #AE


training succesfull
0
1


In [ ]:
#A Set up the perceptron class
#B the __init__ method of any python class is always ran when creating an instance of the class. Some default values are set here 
#C the threshold is an arbitrary cutoff between 0 and 1 to decide whether the prediction becomes a 0 or a 1. often its just 0.5, right in the middle but it depends on the use case.
#D the learning rate of an algorithm is the adjustment it makes every time a new observation comes in. If this is high the model will adjust quickly to new observations but might "overshoot" and never get very precise. 
#oversimplified example: the optimal (and unknown) weight for an x-variable = 0.75. current estimation is 0.4. with a learning rate of 0.5 , the adjustment = 0.5 (learning rate) * 1 (size of error) *  1 (value of x) = 0.5. 0.4 (current weight) + 0.5 (adjustment) = 0.9 (new weight). Instead of 0.75. the adjustment was too big to get the correct result. 
#E x and y variables data are assigned to the class
#F one epoch is one run through all the data. We allow for a maximum of 10 runs until we stop the perceptron. 
#G each observation will end up with a weight. the initialize function sets these weights for each incoming observation. We allow for two options: all weights start at 0 or they are assigned a small (between 0 and 0.05) random weight 
#H the training function
#I we start at the first epoch. one epoch is one run through all the data.
#J true is always true so technically this is a never ending loop but we build in several stop (break) conditions.
#K initiate the number of encountered errors at 0 for each epoch. This is important, if an epoch ends without errors, the algorithm converged and we are done. 
#L add one to the current number of epochs
#M we loop through the data and feed it to the train observation function, one observation at a time
#N if by the end of the epoch we don't have an error, the training was successful
#O if we reach the maximum number of allowed runs, we stop looking for a solution.
#P the train observation function is run for every observation and will adjust the weights using the formula explained earlier.
#Q A prediction is made for this observation. Because its binary this will be either 0 or 1.
#R The real value (y) is either 0 or 1, the prediction is also 0 or 1. If it's wrong we get an error of either 1 or -1
#S In case we have a wrong prediction (an error) we need to adjust the model
#T ad 1 to the error count
#U For every predictor variable in the input vector (X), we will adjust their weight. 
#V adjust the weight for every predictor variable using the learning rate , the error and the actual value of the predictor variable
#W we return the error count because we need to evaluate it at the end of the epoch
#X The predict class
#Y The values of the predictor values is multiplied by their respective weights (this multiplication is done by np.dot). Then the outcome is 
# compared to the overall threshold (here this is 0.5) to see is a 0 or 1 should be predicted.
#Z Our X (predictors) data matrix 
#AA Our y (target) data vector
#AB we instantiate our perceptron class with the data from matrix X and vector y
#AC The weights for the predictors is initialized (as explained higher up)
#AD the perceptron model is trained. It will try to train until it either converges (no more errors) or it runs out of training runs (epochs)
#AE we check what the perceptron would now predict given different values for the predictor variables. In the first case
#it will predict 0, in the second it predicts a 1

#Block matrix calculations

β=〖(X^T X)〗^(-1) X^T y

In [2]:
import dask.array as da
import bcolz as bc
import numpy as np
import dask

n = 1e4 #A

ar = bc.carray(np.arange(n).reshape(n/2,2)  , dtype='float64', rootdir = 'ar.bcolz', mode = 'w') #B
y  = bc.carray(np.arange(n/2), dtype='float64', rootdir = 'yy.bcolz', mode = 'w') #B,

dax = da.from_array(ar, chunks=(5,5)) #C
dy = da.from_array(y,chunks=(5,5)) #C

XTX = dax.T.dot(dax) #D
Xy  = dax.T.dot(dy) #E

coefficients = np.linalg.inv(XTX.compute()).dot(Xy.compute()) #F

coef = da.from_array(coefficients,chunks=(5,5)) #G

ar.flush() #H
y.flush() #H

predictions = dax.dot(coef).compute() #I
print predictions

[  0.00000000e+00   1.00000001e+00   2.00000003e+00 ...,   4.99700007e+03
   4.99800007e+03   4.99900007e+03]


In [ ]:
# A Number of observations (scientific notation). 1e4 = 10.000 , feel free to change this. 
# B create fake data: np.arange(n).reshape(n/2,2) creates a matrix of 5000 by 2 (because we set n to 10.000)
# bc.carray = numpy array extension that can swap to disc. This is also stored in a compressed way)
# rootdir = 'ar.bcolz'  --> create file on disc for in case he goes out of RAM. 
# you can check this on your file system next to this ipython file or wherever location you ran this code from
# mode = 'w' --> write mode
# dtype = 'float64' --> storage type of the data (which are float numbers)
# C Here, block matrices are created for the predictor variables (ar) and target (y). 
# a block matrix is a matrix cut in pieces (the blocks) 
# da.from_array() reads data from disc or RAM (wherever it resides currently)
# chunks=(5,5) every block is a 5 * 5 matrix , (unless < 5 observations or variables are left)
#D The XTX is defined (defining it is "lazy") as the X matrix multiplied with its transposed version.
#This is a building block of the formula to  do linear regression using matrix calculation
#E Xy is the y vector multiplied with the transposed X matrix. Again the matrix is only defined, not calculated yet.
#This is also a building block of the formula todo linear regression using matrix calculation (see formula)
#F the coefficients are calculated using the matrix linear regression function. 
# np.linalg.inv() is the ^(-1) in this function or "inversion" of the matrix
# X.dot(Y) --> multiply the matrix X with another matrix Y
#G the coefficients are also put into a block matrix
#we got a numpy array back from last step so we need to explicitly convert it back to a "da array".
#H flush memory data, it's no longer needed to have large matrices in memory
#I score the model (make predictions) 

#Generating out of memory error

In [14]:
import glob
from sklearn.datasets import load_svmlight_file

files = glob.glob('C:\Users\Gebruiker\Downloads\url_svmlight.tar\url_svmlight\*.svm') #A
files = glob.glob('C:\Users\Gebruiker\Downloads\url_svmlight\url_svmlight\*.svm') #B

print "there are %d files" % len(files) #C
X,y = load_svmlight_file(files[0] ,n_features=3231952) #D
X.todense() #E

there are 121 files


MemoryError: 

In [ ]:
#A point to files (linux)
#B point to files (windows: tar file needs to be untarred first)
#C indication of number of files
#D load in the files
#E The data is a big but sparse matrix. by turning it into a dense matrix (every 0 is represented in the file) we create an out of memory error

##check % of non-zero data

In [9]:
print "number of non-zero entries %2.6f"  % float((X.nnz)/(float(X.shape[0]) * float(X.shape[1])))

number of non-zero entries 0.000036


#Recognising malicious urls Case Study

##Step 1: Research goal

###The goal of this case study is to recognize malicious urls amongst a large set of normal urls

##Step 2: data retrieval

###import libraries

In [1]:
import tarfile
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import load_svmlight_file
import numpy as np

download data from http://sysnet.ucsd.edu/projects/url/#datasets

###import data

In [2]:
uri = 'D:\Python Book\Chapter 4\url_svmlight.tar.gz' #A
tar = tarfile.open(uri,"r:gz")

In [ ]:
#A The uri variable holds the location you saved the downloaded files. You will need to fill this uri variable out yourself for the code to run on your computer

##Step 3: Data Cleansing

### no real data cleansing is necessary here, the urls were pre-cleansed. 

##Step 4: data exploration

###We need to find out the dimensions of our data without uncompressing everything.

In [3]:
max_obs = 0 #B
max_vars = 0 #C
i = 0 #D
split = 5 #E
for tarinfo in tar: #F
    print " extracting %s,f size %s" % (tarinfo.name, tarinfo.size)
    if tarinfo.isfile():
        f = tar.extractfile(tarinfo.name)#G
        X,y = load_svmlight_file(f) #H
        max_vars = np.maximum(max_vars, X.shape[0])#I
        max_obs = np.maximum(max_obs, X.shape[1])#I
        
    if i  > split:
        break  #J
    i+= 1
          
print "max X = %s, max y dimension = %s" % (max_obs, max_vars ) #K

 extracting url_svmlight,f size 0
 extracting url_svmlight/Day33.svm,f size 18674876
 extracting url_svmlight/Day32.svm,f size 18599211
 extracting url_svmlight/Day53.svm,f size 18963938
 extracting url_svmlight/Day20.svm,f size 18633460
 extracting url_svmlight/Day7.svm,f size 18777054
 extracting url_svmlight/Day117.svm,f size 18106370
max X = 3231952, max y dimension = 20000


In [ ]:
#A The uri variable holds the location you saved the downloaded files. You will need to fill this uri variable out yourself for the code to run on your computer
#B we don't know how many observations we have, let's initialize it at 0
#C we don't know how many features we have, let's initialize it at 0
#D stop at the twentieth file (instead of all of them: for demonstration purposes)
#E initialize filecounter at 0
#F All files together take up around 2.05 Gb. The trick here is to leave the data compressed in main memory, only unpack what you need
#G We unpack the files one by one to reduce the memory needed
#H Use a helper function load_svmlight_file() to load a specific file
#I Adjust maximum number of observations and variables when necessary (big file)
#J Stop when we reached 5 files  
#K print results

##Step 5: data modelling

### We create a model to distinguish the malicious from the normal urls.

In [4]:
classes = [-1,1] #A
sgd = SGDClassifier(loss="log")  #B
n_features=3231960 #C
split = 20 #D
i = 0 #E
for tarinfo in tar: #F
    if i  > split:
        break
    if tarinfo.isfile():
        f = tar.extractfile(tarinfo.name)#G
        X,y = load_svmlight_file(f,n_features=n_features) #G
        if i < split:
            sgd.partial_fit(X, y, classes=classes) #H
        if i == split:
            print classification_report(sgd.predict(X),y) #I
    i += 1

             precision    recall  f1-score   support

         -1       0.98      0.98      0.98     14227
          1       0.96      0.95      0.96      5773

avg / total       0.97      0.97      0.97     20000



In [ ]:
#A The target variable can be 1 or -1.  “1”: website is safe to visit, “-1”: website is unsafe 
#B Set up the stochastic gradient classifier
#C we know the number of features from data exploration
#D stop at the twentieth file (instead of all of them: for demonstration purposes)
#E initialize filecounter at 0
#F All files together take up around 2.05 Gb. The trick here is to leave the data compressed in main memory, only unpack what you need
#G We unpack the files one by one to reduce the memory needed
#H Use a helper function load_svmlight_file() to load a specific file
#I third important thing is the online algorithm: it can be fed data points file by file (batches)
#J Stop when we reached 5 files and print results

#Recommender system in database case study

In [7]:
#Remark: 
#To install MySQLdb for windows, run the following script in command line with your envirronement activated:
#conda install --channel https://conda.binstar.org/krisvanneste mysql-python

#Check out following stackoverflow post in case this doesn't work for you or you are using Linux:
#http://stackoverflow.com/questions/26705890/cannot-import-mysqldb-python-windows-8-1
#conda install binstar
#binstar search -t conda mysql-python

#Also: feel free to use SQLalchemy instead of MySQLdb

In [6]:
#Remark 2:
#local MySQL database needs to be setup and a schema needs to be present inside.

##Step 1 : research goal

### The goal of this case study is to recommend movies to people based on their previous viewing behavior

##Step 2: retrieving data

### We will create the data ourselves for this case study so no real data retrieval. 

##Step 3: Data cleansing

###import libraries

In [13]:
import MySQLdb
import pandas as pd

###Create customers

####database connection 

In [29]:
user = 'root' #A
password = 'maiton' #A
database = 'test' #A
mc = MySQLdb.connect('localhost',user,password,database) #A
cursor = mc.cursor() #A

In [8]:
#A First we establish the connection; you’ll need to fill out your own username, password and schemaname

In [30]:
nr_customers = 100 #B
colnames = ["movie%d" %i for i in range(1,33)] #B
pd.np.random.seed(2015) #B
generated_customers = pd.np.random.randint(0,2,32 * nr_customers).reshape(nr_customers,32) #B

data = pd.DataFrame(generated_customers, columns = list(colnames)) #C
data.to_sql('cust',mc, flavor = 'mysql', index = True, if_exists = 'replace', index_label = 'cust_id') #C

C:\Users\Gebruiker\Anaconda\envs\book\lib\site-packages\pandas\core\generic.py:1002: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  dtype=dtype)


In [31]:
data[0:2]

,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10,...,movie23,movie24,movie25,movie26,movie27,movie28,movie29,movie30,movie31,movie32
0,0,0,0,0,1,0,1,0,0,0,...,1,0,1,0,1,1,0,1,1,0
1,0,0,0,1,0,1,1,1,0,0,...,1,1,1,0,1,1,0,1,0,0


In [ ]:
#B Next we simulate a database with customers and create a few observations 
#C Store the data inside a pandas dataframe and write the dataframe in a MySQL table called "cust". If this table already exists, replace it

###Create bit strings

In [38]:
def createNum(x1,x2,x3,x4,x5,x6,x7,x8): #A
    return  [int('%d%d%d%d%d%d%d%d' % (i1,i2,i3,i4,i5,i6,i7,i8),2) 
for (i1,i2,i3,i4,i5,i6,i7,i8) in zip(x1,x2,x3,x4,x5,x6,x7,x8)]

assert int('1111',2) == 15 #B
assert int('1100',2) == 12 #B
assert createNum([1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,0],[1,0]) == [255,252] #B

store = pd.DataFrame() #C
store['bit1'] = createNum(data.movie1, data.movie2,data.movie3,data.movie4,data.movie5,
data.movie6,data.movie7,data.movie8) #C
store['bit2'] = createNum(data.movie9, data.movie10,data.movie11,data.movie12,data.movie13,
data.movie14,data.movie15,data.movie16) #C
store['bit3'] = createNum(data.movie17, data.movie18,data.movie19,data.movie20,data.movie21,
data.movie22,data.movie23,data.movie24) #C
store['bit4'] = createNum(data.movie25, data.movie26,data.movie27,data.movie28,data.movie29,
data.movie30,data.movie31,data.movie32) #C

In [13]:
store[0:2]

,bit1,bit2,bit3,bit4
0,10,62,42,182
1,23,28,223,180


In [ ]:
#A We represent the string as a numeric value. The string will be a concatenation of zeros (0) and ones (1) 
# as these indicate whether someone has seen a certain movie or not. The strings are then regarded as bitcode.
# For example: 0011 is the same as the number 3
# what def createNum() does: take in 8 values, concatenate these 8 column values and turn into a string. Then turn the bytecode of the string into a number

#B Test is the function works correctly
# binary code 1111 is the same as 15 (=1*8+1*4+1*2+1*1)
# if the assert fails, it will raise an assert error, otherwise nothing will happen.

#C Translate the movie column to 4 bit-strings in numeric form. each bitstring represents 8 movies. 4 *8 = 32 movies.
#REMARK: you could simply use a 32bit string instead of 4*8  (its to keep the code short)

###Create hash functions

In [39]:
def hash_fn(x1,x2,x3): #A
    return [b'%d%d%d' % (i,j,k) for (i,j,k) in zip(x1,x2,x3)]

assert hash_fn([1,0],[1,1],[0,0]) == [b'110',b'010'] #B

store['bucket1'] = hash_fn(data.movie10, data.movie15,data.movie28) #C
store['bucket2'] = hash_fn(data.movie7, data.movie18,data.movie22) #C
store['bucket3'] = hash_fn(data.movie16, data.movie19,data.movie30) #C
store.to_sql('movie_comparison',mc, flavor = 'mysql', index = True, 
             index_label = 'cust_id', if_exists = 'replace') #D

In [18]:
store[0:2]

,bit1,bit2,bit3,bit4,bucket1,bucket2,bucket3
0,10,62,42,182,011,100,011
1,23,28,223,180,001,111,001


In [ ]:
#A Define the hash function (it is exactly like the createNum() function without the final conversion to a number and for 3 columns instead of 8)
#B Test if it works correctly (if no error is raised, it works)
#it's sampling on columns but all observations will be selected
#C Create hash values from customer movies, respectively [10,5,28],[7,18,22],[16,19,30]
#D store this information in the database

##Create an index

In [40]:
def createIndex(column, cursor): #A
    sql = 'CREATE INDEX %s ON movie_comparison (%s);' % (column, column)
    cursor.execute(sql)

createIndex('bucket1',cursor)  #B
createIndex('bucket2',cursor)  #B
createIndex('bucket3',cursor)  #B

In [ ]:
#A Create a function to easily create indices. indices will quicken retrieval
#B actually put the index on the bit buckets

##Step 4: data exploration

###This step would be pretty redundent since we created the data ourselve.

##Step 5: model building

###Create hamming distance

In [32]:
Sql = '''
CREATE FUNCTION HAMMINGDISTANCE(
  A0 BIGINT, A1 BIGINT, A2 BIGINT, A3 BIGINT, 
  B0 BIGINT, B1 BIGINT, B2 BIGINT, B3 BIGINT
)

RETURNS INT DETERMINISTIC
RETURN 
  BIT_COUNT(A0 ^ B0) +
  BIT_COUNT(A1 ^ B1) +
  BIT_COUNT(A2 ^ B2) +
  BIT_COUNT(A3 ^ B3); '''
#A
cursor.execute(Sql) #B

Sql = '''Select hammingdistance(
    b'11111111',b'00000000',b'11011111',b'11111111'
,b'11111111',b'10001001',b'11011111',b'11111111'
)''' #C
pd.read_sql(Sql,mc) #D

OperationalError: (1304, 'FUNCTION HAMMINGDISTANCE already exists')

In [ ]:
#A Define the function. It takes 8 input arguments: 4 strings of length 8 for the first customer and another 4 strings of length 8 for the second customer. This way we can compare 2 customers side by side for 32 movies.
#B The function is stored in database. You can only do this once, running this code a  second time will result in an error: OperationalError: (1304, 'FUNCTION HAMMINGDISTANCE already exists') 
#C To check this function you can run this SQL statement with 8 fixed strings. notice the "b" before each string, indicating that you’re passing bit values. The outcome of this particular test should be 3 which indicates the series of strings differ in only 3 places.
#D this actually runs the query.

##Step 6: presentation & automatisation

### At this point we have an application that use the database to lookup a customer and present him with movies 

###Finding similar customers

In [20]:
customer_id = 27 #A
sql = "select * from movie_comparison where cust_id = %s" % customer_id  #A
cust_data = pd.read_sql(sql,mc) #A
sql =  """ select cust_id,hammingdistance(bit1,
bit2,bit3,bit4,%s,%s,%s,%s) as distance
           from movie_comparison where bucket1 = '%s' or bucket2 ='%s' 
or bucket3='%s' order by distance limit 3""" % (cust_data.bit1[0],cust_data.bit2[0], 
cust_data.bit3[0], cust_data.bit4[0],
       cust_data.bucket1[0], cust_data.bucket2[0],cust_data.bucket3[0])#B
shortlist = pd.read_sql(sql,mc) #C
shortlist

,cust_id,distance
0,27,0
1,2,8
2,97,9


In [ ]:
#A pick a customer from database
#B we do 2-step sampling :
# first sampling: Index must be exactly the same as the one of the selected customer (is based on 9 movies). selected people must have seen (or not seen) these 9 movies
# like our customer did. 
# second is a ranking based on the 4 bit strings. These take into account all the movies in the database. 
#C we show the 3 customers most like customer 27. Of course customer 27 ends up first.

###Finding an unseen movie

In [21]:
cust = pd.read_sql('select * from cust where cust_id in (27,2,97)',mc)#A
dif = cust.T #B
dif[dif[0] != dif[1]] #C
#A Select movies that customer 27,2,97 have seen
#B Transpose for convenience
#C Select movies that customer 27 didn’t see yet 

,0,1,2
cust_id,2,27,97
movie3,0,1,1
movie9,0,1,1
movie11,0,1,1
movie12,1,0,0
movie15,1,0,0
movie16,0,1,1
movie25,0,1,1
movie31,1,0,0
